In [1]:
import open_flamingo
from open_flamingo import create_model_and_transforms
from huggingface_hub import hf_hub_download
import torch
from PIL import Image
import requests


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [21]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("mosaicml/mpt-1b-redpajama-200b", trust_remote_code=True, cache_dir="/mnt/d/models/")
model.to(device='cuda:0', dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("mosaicml/mpt-1b-redpajama-200b")

/home/tovitu/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-1b-redpajama-200b/a0929818043de593a6b4248481e18a4b276f5e1e/attention.py:289: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn(


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


In [30]:
from scripts.datasets import SQUAD_dataset

dataset = SQUAD_dataset()

In [23]:
token = tokenizer(data.val_dataset[0]['context'])

In [26]:
output = model.generate(
    input_ids = torch.tensor([token['input_ids']]).to(0),
    attention_mask = torch.tensor([token['attention_mask']], dtype=torch.bfloat16).to(0)
)

/home/tovitu/miniconda3/envs/flamingo/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/tovitu/miniconda3/envs/flamingo/lib/python3.10/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 158, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [41]:
import tqdm
import os
import json

output_dir = '/mnt/d/datasets/'

device = 0
precision = torch.bfloat16
predictions = []
references = []

counter = 0
for idx in tqdm.tqdm(range(2000)):
    data = dataset.val_dataset[idx]

    context = data["context"]
    question = data["question"]

    context_text = dataset.make_in_context(dataset.train_dataset, shots=4)

    # Encode text
    prompt = context_text +  dataset.qa_prompt(context, question)
    text_token = tokenizer(prompt)

    answer_len = max(
        [len(answer.split()) for answer in data["answers"]["text"]]
    )
    if os.environ.get("MAX_LEN") != None:
        answer_len = int(os.environ["MAX_LEN"])

    answer_len = 20

    # Inference
    output = model.generate(
        input_ids = torch.tensor([text_token["input_ids"]]).to(device),
        attention_mask = torch.tensor([text_token["attention_mask"]]).to(
            device=0, dtype=torch.bfloat16
        ),
        max_new_tokens=answer_len,
        pad_token_id=50277,
    )
    output = tokenizer.decode(output[0])
    output = output[len(prompt) :]
    output = output.replace("<|endofchunk|>", "")
    output = dataset.postprocess_vqa_generation(output)
    output = dataset.normalize(output)

    # Put together predictions
    predictions.append({"prediction_text": output, "id": data["id"]})
    references.append({"answers": data["answers"], "id": data["id"]})

with open(output_dir + "squad_resutls.json", "w") as f:
    f.write(json.dumps(predictions, indent=4))
with open(output_dir + "squad_references.json", "w") as f:
    f.write(json.dumps(references, indent=4))

  2%|▏         | 40/2000 [00:49<40:04,  1.23s/it]


KeyboardInterrupt: 

In [ ]:
from evaluate import load

with open(f"{output_dir}squad_resutls.json", "r") as f:
    predictions = json.load(f)
with open(f"{output_dir}squad_references.json", "r") as f:
    references = json.load(f)

squad_metric = load("squad")
results = squad_metric.compute(predictions=predictions, references=references)
print(results)


{'exact_match': 20.85, 'f1': 35.68241137623221}
